In [ ]:
import numpy as np 
import os 
import pandas as pd
import tensorflow
dir_path = './input'

train_dir = os.path.join(dir_path,'train.csv')
test_dir = os.path.join(dir_path,'test.csv')

In [ ]:
train_csv = pd.read_csv(train_dir)
test_csv = pd.read_csv(test_dir)
train_csv.head()
train_csv.shape

In [ ]:
split_df = pd.DataFrame([train_csv['pixels'][i].split() for i in range(train_csv.shape[0])])

In [ ]:
train_joined = train_csv.join(split_df)

In [ ]:
train_joined.drop('pixels',axis = 1,inplace = True)

In [ ]:
Y_train = train_joined['emotion']

In [ ]:
X_train = train_joined.iloc[:,1:2305]

In [ ]:
import matplotlib
import matplotlib.pyplot as plt 

x_train_raw = X_train.to_numpy()
y_train_raw = Y_train.to_numpy()

x_train = x_train_raw.astype(np.float64)
y_train = y_train_raw.astype(np.int)

x_train/=255

In [ ]:
# Showing a couple of images taken from the train data set
fig = plt.figure()
ax1 = fig.add_subplot(221)
ax1.imshow(x_train[0].reshape((48,48)))
ax2 = fig.add_subplot(222)
ax2.imshow(x_train[1].reshape((48,48)))
ax3 = fig.add_subplot(223)
ax3.imshow(x_train[2].reshape((48,48)))
ax4 = fig.add_subplot(224)
ax4.imshow(x_train[3].reshape((48,48)))
plt.show()

In [ ]:
# Looking at the histogram of the emotions 
#plt.hist(range(len(np.unique(y_train))),y_train)

plt.hist(y_train_raw,range(6))

# Emotion nb 1 seems to be very underrepresented, 
# with that lack of data it is most likely we will perform poorly on recognizing those

In [ ]:
print(x_train_raw.shape)
print(y_train_raw.shape)

In [ ]:
# We build a train/test data set
# We have a lot of data (overall) so we can afford to take a relatively large validation set
from sklearn.model_selection import train_test_split

x_train,x_val, y_train,y_val = train_test_split(x_train,y_train,test_size = 0.2,stratify = y_train_raw)

In [ ]:
print(x_train.shape)
print(y_train.shape)
print(y_val.shape)

In [ ]:
plt.hist(y_val,range(6),label='histogram of the validation set')
plt.hist(y_train,range(6),fc= (0,0,1,0.5),label='histogram of the training set')
plt.legend()
plt.xlabel('Classes')
plt.ylabel('Instances')
plt.show()

In [ ]:
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier()
rf.fit(x_train, y_train)

In [ ]:
pred = rf.predict(x_val)

In [ ]:
from sklearn.metrics import accuracy_score

score = accuracy_score(pred,y_val)
# 44 % Accuracy
print('Accuracy of the random forest classifier :',score)

# We have managed to get better than a random classifier (around 1/7)
# This is a good baseline model

# We use it to show the importance of the pixels 

fi = rf.feature_importances_
plt.imshow(np.reshape(fi,(48,48)))
plt.show()

# Showing a couple of images taken from the train data set
fig = plt.figure()
ax1 = fig.add_subplot(221)
ax1.imshow(x_train[np.random.randint(15)].reshape((48,48)))
ax2 = fig.add_subplot(222)
ax2.imshow(x_train[np.random.randint(15)].reshape((48,48)))
ax3 = fig.add_subplot(223)
ax3.imshow(x_train[np.random.randint(15)].reshape((48,48)))
ax4 = fig.add_subplot(224)
ax4.imshow(x_train[np.random.randint(15)].reshape((48,48)))
plt.show()

Interesting fact, the model seems to look for clues mainly around the mouth, gives importance to the cheeks, and also a bit to the forehead. Pretty much what we do as humans ! 

In [ ]:
# We will now train a convnet (duh) to tacke the problem

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Dropout, Flatten, BatchNormalization
from tensorflow.keras.utils import to_categorical

In [ ]:
x_train = np.reshape(x_train,(x_train.shape[0],48,48,1))
x_val = np.reshape(x_val,(x_val.shape[0],48,48,1))

In [ ]:
y_train = to_categorical(y_train)
y_val = to_categorical(y_val)

# MODEL 1

In [ ]:
import tensorflow
model = Sequential()

model.add(Conv2D(32,(3,3),activation = 'relu', padding = 'same',input_shape = (48,48,1)))
model.add(MaxPooling2D(2,2))
model.add(Dropout(0.3))
model.add(Conv2D(64,(3,3),activation = 'relu', padding = 'same'))
model.add(MaxPooling2D(2,2))
model.add(Dropout(0.3))
model.add(Conv2D(128,(3,3),activation = 'relu', padding = 'same'))
model.add(MaxPooling2D(2,2))
model.add(Dropout(0.3))
model.add(Conv2D(128,(3,3),activation = 'relu', padding = 'same'))
model.add(MaxPooling2D(2,2))
model.add(Dropout(0.3))
model.add(Conv2D(128,(3,3),activation = 'relu', padding = 'same'))
model.add(MaxPooling2D(2,2))
model.add(Dropout(0.3))
model.add(Flatten())
model.add(Dense(128,activation='relu'))
model.add(Dense(7,activation='softmax'))
model.summary()

In [ ]:
model.compile(optimizer='rmsprop',loss='categorical_crossentropy',metrics=['accuracy'])
history = model.fit(x_train,y_train,validation_data = (x_val,y_val),epochs = 50, batch_size = 128)

In [ ]:
# First model, only a bit better than the random forest 
# Lets see what's wrong with it

# A first obvious thing is the fact that is not complex enough to learn the training set
# We are quite obviously under fitting

# Lets try to make the model a bit more complex to make the hypothesis space larger 

In [ ]:
loss = history.history['loss']
val_loss = history.history['val_loss']
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

epochs = range(1,len(loss)+1)

plt.plot(epochs,loss,'bo',label='training loss')
plt.plot(epochs,val_loss,'b',label='val loss')
plt.legend()
plt.xlabel('epochs')
plt.ylabel('losses')
plt.show()

plt.plot(epochs,acc,'bo',label='training acc')
plt.plot(epochs,val_acc,'b',label='val acc')
plt.legend()
plt.xlabel('epochs')
plt.ylabel('accuracies')
plt.show()


# MODEL 2 

In [ ]:
model_2 = Sequential()

model_2.add(Conv2D(256,(3,3),activation = 'relu', padding = 'same',input_shape = (48,48,1)))
model_2.add(MaxPooling2D(2,2))
model_2.add(Dropout(0.3))


model_2.add(Conv2D(256,(5,5),activation = 'relu', padding = 'same'))
model_2.add(MaxPooling2D(2,2))
model_2.add(Dropout(0.3))


model_2.add(Conv2D(256,(5,5),activation = 'relu', padding = 'same'))
model_2.add(MaxPooling2D(2,2))
model_2.add(Dropout(0.3))


model_2.add(Conv2D(256,(5,5),activation = 'relu', padding = 'same'))
model_2.add(MaxPooling2D(2,2))
model_2.add(Dropout(0.3))


model_2.add(Flatten())
model_2.add(Dense(256,activation='relu'))
model_2.add(Dense(256,activation='relu'))
model_2.add(Dense(7,activation='softmax'))
model_2.summary()

model_2.compile(optimizer='rmsprop',loss='categorical_crossentropy',metrics=['accuracy'])
history = model_2.fit(x_train,y_train,validation_data = (x_val,y_val),epochs = 20, batch_size = 128,verbose=1)

loss = history.history['loss']
val_loss = history.history['val_loss']
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

epochs = range(1,len(loss)+1)

plt.plot(epochs[1:],loss[1:],'bo',label='training loss')
plt.plot(epochs[1:],val_loss[1:],'b',label='val loss')
plt.legend()
plt.xlabel('epochs')
plt.ylabel('losses')
plt.show()

plt.plot(epochs,acc,'bo',label='training acc')
plt.plot(epochs,val_acc,'b',label='val acc')
plt.legend()
plt.xlabel('epochs')
plt.ylabel('accuracies')
plt.show()


In [ ]:
# The model is severely over fitting, so we'll try to tune the first one a bit 

# We now use a pretrained model, facenet

In [ ]:
m_pre = Sequential()
from tensorflow.keras.applications import InceptionResNetV2

# Since ResNetV2 expects 3 input channels we repeat the image

x_train_3c = x_train[:,:,:,:,np.newaxis]
x_train_3c.shape
x_train_rgb = np.repeat(x_train, 3, -1)

plt.imshow(x_train_rgb[0,:,:,0].reshape(48,48))
plt.show()
plt.imshow(x_train_rgb[0,:,:,1].reshape(48,48))
plt.show()
plt.imshow(x_train_rgb[0,:,:,2].reshape(48,48))
plt.show()

x_val_rgb = np.repeat(x_val,3,-1)

# It worked

In [ ]:
print(x_val_rgb.shape)
print(x_train_rgb.shape)

In [ ]:
conv_base = InceptionResNetV2(include_top = False,weights ='imagenet')

In [ ]:
conv_base.trainable = False

In [ ]:
model = Sequential()
model.add(tensorflow.keras.layers.ZeroPadding2D((75,75), input_shape = (48,48,3)))

In [ ]:
model.add(conv_base)
model.add(Flatten())
model.add(Dense(128, activation ='relu'))
model.add(Dense(7,activation='softmax'))

model.summary()

model.compile(optimizer='rmsprop',metrics = ['accuracy'],loss='categorical_crossentropy')

In [ ]:
history = model.fit(x_train_rgb,y_train, validation_data = (x_val_rgb,y_val), epochs = 20, batch_size = 256)

In [ ]:
# the model is very, very long to train, and I don't have the ressources to experiment on it, so we'll keep the handmade one

# Evaluation



In [ ]:
# We train the model on the validation + training data set

In [ ]:
x_train_full = np.concatenate((x_train,x_val))

In [ ]:
y_train_full = np.concatenate((y_train,y_val))
y_train_full.shape

In [ ]:
full = os.path.join(dir_path, 'icml_face_data.csv')

full_csv = pd.read_csv(full)

In [ ]:
cop[full_csv['Usage'] != 'Training']

In [ ]:
testing_csv = full_csv[full_csv['Usage'] == 'PublicTest']
private_csv = full_csv[full_csv['Usage'] == 'PrivateTest']

In [ ]:
testing_csv.drop('Usage',axis=1,inplace=True)
private_csv.drop('Usage',axis=1,inplace=True)

In [ ]:
testing_csv.reset_index(inplace=True)
private_csv.reset_index(inplace=True)

In [ ]:
def convert_tests(df):
    
    # We plit the data set 
    split_df_test = pd.DataFrame([df['pixels'][i].split() for i in range(df.shape[0])])
    temp_joined = df.join(split_df_test)
    
    # We drop the pixels column
    temp_joined.drop('pixels',axis = 1,inplace = True)
    
    # We get the X : pixels 
    X = temp_joined.iloc[:,1:2305]
    x_raw = X.to_numpy()
    x = x_raw.astype(np.float64)
    x/=255

    # We get the Y : labels
    Y = temp_joined['emotion']
    y_raw = Y.to_numpy()
    y = y_raw.astype(np.int)
    y = to_categorical(y)

    x = np.reshape(x,(x.shape[0],48,48,1))
    
    print('Shape of attributes matrix',x.shape)
    print('Shape of label matrix',y.shape)
    
    return x,y

In [ ]:
x_public, y_public = convert_tests(testing_csv)
x_private,y_private = convert_tests(private_csv)

In [ ]:
plt.imshow(x_public[2].reshape(48,48))
plt.show()
y_public[2]

In [ ]:
import tensorflow
model = Sequential()

model.add(Conv2D(32,(3,3),activation = 'relu', padding = 'same',input_shape = (48,48,1)))
model.add(MaxPooling2D(2,2))
model.add(Dropout(0.3))
model.add(Conv2D(64,(3,3),activation = 'relu', padding = 'same'))
model.add(MaxPooling2D(2,2))
model.add(Dropout(0.3))
model.add(Conv2D(128,(3,3),activation = 'relu', padding = 'same'))
model.add(MaxPooling2D(2,2))
model.add(Dropout(0.3))
model.add(Conv2D(128,(3,3),activation = 'relu', padding = 'same'))
model.add(MaxPooling2D(2,2))
model.add(Dropout(0.3))
model.add(Conv2D(128,(3,3),activation = 'relu', padding = 'same'))
model.add(MaxPooling2D(2,2))
model.add(Dropout(0.3))
model.add(Flatten())
model.add(Dense(128,activation='relu'))
model.add(Dense(7,activation='softmax'))
model.summary()

model.compile(optimizer='rmsprop',loss='categorical_crossentropy',metrics=['accuracy'])

history = model.fit(x_train_full,y_train_full,epochs = 150, batch_size = 128)

loss = history.history['loss']

acc = history.history['accuracy']

epochs = range(1,len(loss)+1)

plt.plot(epochs,loss,'bo',label='training loss')
plt.plot(epochs,acc,'b',label='training acc')
plt.legend()
plt.xlabel('epochs')
plt.ylabel('losses')
plt.show()

In [ ]:
model.evaluate(x_private,y_private)

# My Model Accuracy : 62.41 % (10/58)

In [ ]:
model.save('model_facial_recognition.h5')